In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"  # Optional, may help with fragmentation

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
        print(f"Could not set memory growth: {e}")

2025-07-03 17:41:24.066832: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 17:41:24.083465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751578884.100693 2515541 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751578884.105993 2515541 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751578884.119687 2515541 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

# Set project root directory and add `src` to path
PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import numpy as np
import tensorflow as tf

from word2gm_fast.models.config import Word2GMConfig
from word2gm_fast.training.notebook_training import run_notebook_training
from word2gm_fast.utils.tfrecord_io import load_pipeline_artifacts
from word2gm_fast.utils.resource_summary import print_resource_summary

In [3]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: gv006.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Requested partitions: v100,rtx8000,a100_2,a100_1,h100_1
   Running on: SSH failed: Host key verification failed.
   Job ID: 63356746
   Node list: gv006

GPU Information:
   CUDA GPUs detected: 1
   GPU 0: Tesla V100-PCIE-32GB
      Memory: 0.3/32.0 GB (31.7 GB free)
      Temperature: 32°C
      Utilization: GPU 0%, Memory 0%

TensorFlow GPU Detection:
   TensorFlow detects 1 GPU(s)
      /physical_device:GPU:0, Memory growth: True
   Built with CUDA: True
============================================================</pre>

In [6]:
# Data loading and pipeline setup

# Define paths for your corpus artifacts and output
dataset_artifacts_dir = (
    '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/'
    '1940_artifacts'
)
output_dir = '/scratch/edk202/word2gm-fast/output/test_corpus'
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Set TensorBoard log directory
tensorboard_log_dir = output_dir + '/tensorboard'

# Load pipeline artifacts (vocab, triplets, etc.)
artifacts = load_pipeline_artifacts(dataset_artifacts_dir)
vocab_table = artifacts['vocab_table']
vocab_list = artifacts['vocab_list']
triplets_ds = artifacts['triplets_ds']
vocab_size = artifacts['vocab_size']

# Build the dataset pipeline: cache -> shuffle -> batch -> prefetch
triplets_ds = triplets_ds.cache()
BATCH_SIZE = 1024 * 6
SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 10
triplets_ds = triplets_ds.shuffle(SHUFFLE_BUFFER_SIZE)
triplets_ds = triplets_ds.batch(BATCH_SIZE)
triplets_ds = triplets_ds.prefetch(tf.data.AUTOTUNE)

print(f'Loaded vocab_size: {vocab_size}')

<pre>Loading pipeline artifacts from: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1940_artifacts</pre>

<pre>Loading vocabulary TFRecord from: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1940_artifacts/vocab.tfrecord</pre>

2025-07-03 17:43:21.203757: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<pre>Loading vocabulary TXT file from: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1940_artifacts/vocab.txt</pre>

<pre>Loading triplet TFRecord from: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1940_artifacts/triplets.tfrecord</pre>

<pre>Triplet TFRecord loaded and parsed</pre>

<pre>All artifacts loaded successfully!</pre>

Loaded vocab_size: 42401


In [ ]:
config = Word2GMConfig(
    vocab_size=vocab_size,
    embedding_size=200,
    num_mixtures=1,
    spherical=True,
    norm_cap=5.0,
    lower_sig=0.05,
    upper_sig=1.0,
    var_scale=0.05,
    loss_epsilon=1e-8,
    wout=True,
    max_pe=False,
)

run_notebook_training(
    training_dataset=triplets_ds,
    save_path=output_dir,
    vocab_size=config.vocab_size,
    embedding_size=config.embedding_size,
    num_mixtures=config.num_mixtures,
    spherical=config.spherical,
    learning_rate=0.05,
    epochs=10,
    adagrad=True,
    normclip=True,
    norm_cap=config.norm_cap,
    lower_sig=config.lower_sig,
    upper_sig=config.upper_sig,
    var_scale=config.var_scale,
    loss_epsilon=config.loss_epsilon,
    wout=config.wout,
    tensorboard_log_path=tensorboard_log_dir,
    monitor_interval=0.5,
    profile=False,
)


**Word2GM Training Hyperparameters:**

| Parameter         | Value                |
|-------------------|----------------------|
| Vocab size        | `42401`  |
| Embedding size    | `200` |
| Mixtures          | `1` |
| Spherical         | `True`   |
| Learning rate     | `0.05` |
| Epochs            | `10` |
| Adagrad           | `True`     |
| Normclip          | `True`    |
| Norm cap          | `5.0`    |
| Lower sigma       | `0.05`   |
| Upper sigma       | `1.0`   |
| Wout              | `True`        |
| Var scale         | `0.05`   |
| Loss epsilon      | `1e-08`|


**Starting epoch 1/10...**

I0000 00:00:1751579005.819620 2515985 service.cc:152] XLA service 0x14faa40149b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751579005.819652 2515985 service.cc:160]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2025-07-03 17:43:25.826171: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751579005.847950 2515985 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1751579005.986270 2515985 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [ ]:
# Launch TensorBoard in the notebook
%load_ext tensorboard
%tensorboard --logdir $tensorboard_log_dir --port 6006

In [ ]:
# --- Save model weights in Keras 3-compatible format ---
# After training, re-instantiate the model and save weights with a supported extension
from word2gm_fast.models.word2gm_model import Word2GMModel
model = Word2GMModel(config)

# Build the model by calling it on a dummy input (tuple of three tensors)
dummy_input = (
    tf.zeros([1], dtype=tf.int32),  # word_ids
    tf.zeros([1], dtype=tf.int32),  # pos_ids
    tf.zeros([1], dtype=tf.int32),  # neg_ids
)
model(dummy_input)

model.load_weights(output_dir + '/model_weights_epoch10.weights.h5')
model.save_weights(output_dir + '/model.weights.h5')

In [ ]:
# Find nearest neighbors for a given word using Word2GMModel and vocab_list
model = Word2GMModel(config)

# Build the model by calling it on a dummy input (tuple of three tensors)
dummy_input = (
    tf.zeros([1], dtype=tf.int32),  # word_ids
    tf.zeros([1], dtype=tf.int32),  # pos_ids
    tf.zeros([1], dtype=tf.int32),  # neg_ids
)
model(dummy_input)

model.load_weights(output_dir + '/model.weights.h5')

# Choose a query word and get its index
query_word = 'happy'  # Change this to any word in your vocab
try:
    query_idx = vocab_list.index(query_word)
except ValueError:
    raise ValueError(f'Word "{query_word}" not found in vocab_list.')

# Get nearest neighbor indices (returns indices, distances or a list of (index, distance) pairs)
result = model.get_nearest_neighbors(query_idx, k=10)
print("Result type:", type(result))
print("Result:", result)

# Try to unpack if possible, else treat as list of pairs
try:
    neighbor_indices, neighbor_distances = result
    neighbors = [(vocab_list[i], float(d)) for i, d in zip(neighbor_indices, neighbor_distances)]
except Exception:
    neighbors = [(vocab_list[i], float(d)) for i, d in result]

print(f'Nearest neighbors for "{query_word}":')
for word, dist in neighbors:
    print(f'{word}\t{dist:.4f}')

In [ ]:
# Inspect the distribution of learned means for a few random words
import numpy as np

# Get the means tensor (shape: [vocab_size, num_mixtures, embedding_size])
means = model.mus.numpy()  # or model.mus_out if wout=True and use_output

print("Means shape:", means.shape)
for idx in np.random.choice(means.shape[0], size=5, replace=False):
    print(f"Word: {vocab_list[idx]}")
    print("  Mean (first mixture):", means[idx, 0, :5])  # print first 5 dims
    print("  Mean std (all mixtures):", means[idx].std())